In this file I create:

1. The connection between PostgreSQL and Python

2. The database called **etl_workshop_first**.

3. Two tables with its properties: **raw_applicant** and **applicant**.

In [23]:
pip install -r code/requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Import libraries:

In [24]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from code.connect_database import Connection_Postgres

## Let's know the table
This table has 50.000 rows and 10 columns.

In [25]:
# Create connection with PostgreSQL
connection = Connection_Postgres()
cursor = connection.connection.cursor()
# Consult data
query_to_do = "SELECT first_name, last_name, email, applicant_date, country, experience_year, seniority, technology, code_challenge_score, technical_interview_score FROM raw_applicant"
cursor.execute(query_to_do)
record_table = cursor.fetchall()
# Get column names
column_names = [desc[0] for desc in cursor.description]
# Create dataframe
dataframe = pd.DataFrame(record_table, columns=column_names)
# Create connection with engine
connection_string = f"postgresql://{connection.connection_config['user']}:{connection.connection_config['password']}@{connection.connection_config['host']}:{connection.connection_config['port']}/{connection.connection_config['database']}"
postgres_engine = create_engine(connection_string)
# Close connection
connection.close_connection_database()

In [26]:
# Check info
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   first_name                 50000 non-null  object
 1   last_name                  50000 non-null  object
 2   email                      50000 non-null  object
 3   applicant_date             50000 non-null  object
 4   country                    50000 non-null  object
 5   experience_year            50000 non-null  int64 
 6   seniority                  50000 non-null  object
 7   technology                 50000 non-null  object
 8   code_challenge_score       50000 non-null  int64 
 9   technical_interview_score  50000 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 3.8+ MB


Describing the table:

In [27]:
# Describe data
dataframe.describe()

,experience_year,code_challenge_score,technical_interview_score
count,50000.000000,50000.000000,50000.000000
mean,15.286980,4.996400,5.003880
std,8.830652,3.166896,3.165082
min,0.000000,0.000000,0.000000
25%,8.000000,2.000000,2.000000
50%,15.000000,5.000000,5.000000
75%,23.000000,8.000000,8.000000
max,30.000000,10.000000,10.000000


Checking for NaN values that I couldn't see

In [28]:
dataframe.isna().sum()

first_name                   0
last_name                    0
email                        0
applicant_date               0
country                      0
experience_year              0
seniority                    0
technology                   0
code_challenge_score         0
technical_interview_score    0
dtype: int64

Checking values for each column of table

In [29]:
for column in dataframe.columns:
    print(dataframe[column].value_counts())
    print("-"*10)

first_name
Sarai        33
Elyse        32
Eva          31
Anabel       31
Nayeli       30
             ..
Marie         6
Janis         6
Antonetta     5
Paul          5
Gudrun        5
Name: count, Length: 3007, dtype: int64
----------
last_name
Murazik       138
Nitzsche      134
Lang          133
Metz          133
Harvey        133
             ... 
Rutherford     83
Howe           82
White          79
McClure        78
Bechtelar      76
Name: count, Length: 474, dtype: int64
----------
email
marianne31@yahoo.com              3
fern70@gmail.com                  3
benny35@yahoo.com                 2
easter75@gmail.com                2
natalie51@hotmail.com             2
                                 ..
jett.runolfsson@yahoo.com         1
adaline_powlowski@hotmail.com     1
ariel93@hotmail.com               1
magdalen.ankunding17@yahoo.com    1
pattie92@gmail.com                1
Name: count, Length: 49833, dtype: int64
----------
applicant_date
2020-07-07    50
2018-12-06    48
2

I see that in two score columns (code_challenge_score and technical_interview_score) have values among 0 and 10, where 10 is the maximum qualitification.

#### Time analysis
Let's go deeper on applicant_date column.

In [30]:
dataframe['applicant_date'] = pd.to_datetime(dataframe['applicant_date'], format='mixed')
dataframe['applicant_year'] = dataframe['applicant_date'].dt.year
dataframe['applicant_month_name'] = dataframe['applicant_date'].dt.month_name()
dataframe['applicant_month'] = dataframe['applicant_date'].dt.month

In [31]:
dataframe['applicant_year'].value_counts()

applicant_year
2020    11237
2018    11061
2021    11051
2019    11009
2022     5642
Name: count, dtype: int64

In [32]:
dataframe['applicant_month_name'].value_counts()

applicant_month_name
May          4752
March        4649
January      4618
June         4566
April        4542
February     4285
July         3996
October      3813
August       3783
December     3777
September    3610
November     3609
Name: count, dtype: int64

### Let's create the new column of is hiring or not
I take code_challenge_score and technical_interview_score for this.

In [33]:
dataframe['is_hire'] = np.where((dataframe['code_challenge_score'] >= 7) & (dataframe['technical_interview_score'] >= 7), 1, 0)

### Let's load this dataframe to applicant table in PostgreSQL

In [34]:
# Load data obtained to PostgreSQL
dataframe.to_sql('applicant', postgres_engine, if_exists='replace', index=False)
connection.log('Data loaded to {}: {} rows - {} columns.' .format('applicant', dataframe.shape[0], dataframe.shape[1]))